In [6]:
# Origem: Centro de Curitiba (longitude, latitude)
origem = [(-49.2733,-25.4284)]

clientes = [
    (-49.2597, -25.3930),  # Jardim Botânico
    (-49.2673, -25.4404),  # Ópera de Arame
    (-49.2670, -25.4230),  # Museu Oscar Niemeyer
    (-49.2830, -25.4170),  # Parque Barigui
    (-49.2943, -25.4093),  # Parque Tanguá
    (-49.2647, -25.3902),  # Bosque Alemão
    (-49.2722, -25.4354),  # Mercado Municipal
    (-49.3027, -25.4503),  # Torre Panorâmica
    (-49.3213, -25.4184),  # Bosque do Papa
    (-49.2767, -25.4442)   # Catedral Basílica Menor de Nossa Senhora da Luz
]

# Todas as localidades (origem + clientes)
localidades = origem + clientes

In [7]:
import requests

def get_distance_matrix(localidades, origem=None, clientes=None, tipo="int"):
    """
    Obtém a matriz de distâncias e tempos entre as localidades usando o OSRM.

    Args:
        localidades (list): Lista de coordenadas no formato [longitude, latitude].
        origem (int): Índice da localidade de origem (opcional).
        clientes (list): Lista de índices das localidades de destino (opcional).
        tipo (str): Tipo de conversão dos valores ("int" ou "float").

    Returns:
        duration_matrix (list): Matriz de tempos em segundos.
        distance_matrix (list): Matriz de distâncias em metros.
    """
    # Formata as coordenadas para o formato "longitude,latitude"
    coordinates = ";".join([f"{loc[0]},{loc[1]}" for loc in localidades])

    # Define as origens e destinos
    if origem is not None:
        sources = [origem]  # Apenas a origem especificada
    else:
        sources = list(range(len(localidades)))  # Todas as localidades como origens

    if clientes is not None:
        destinations = clientes  # Apenas os clientes especificados
    else:
        destinations = list(range(len(localidades)))  # Todas as localidades como destinos

    # Configura os parâmetros da requisição
    params = {
        "sources": ";".join(map(str, sources)),  # Índices das origens
        "destinations": ";".join(map(str, destinations)),  # Índices dos destinos
        "annotations": "duration,distance"  # Solicita tempos e distâncias
    }

    # Faz a requisição ao OSRM
    url = "http://router.project-osrm.org/table/v1/driving/"
    response = requests.get(url + coordinates, params=params)

    # Verifica se a requisição foi bem-sucedida
    if response.status_code == 200:
        data = response.json()
        duration_matrix = data['durations']  # Matriz de tempos em segundos
        distance_matrix = data['distances']  # Matriz de distâncias em metros

        # Converte os valores para o tipo especificado
        if tipo == "int":
            duration_matrix = [[int(val) for val in row] for row in duration_matrix]
            distance_matrix = [[int(val) for val in row] for row in distance_matrix]
        elif tipo == "float":
            duration_matrix = [[float(val) for val in row] for row in duration_matrix]
            distance_matrix = [[float(val) for val in row] for row in distance_matrix]
        else:
            raise ValueError("Tipo inválido. Use 'int' ou 'float'.")

        return duration_matrix, distance_matrix
    else:
        print("Erro ao obter a matriz de distâncias:", response.status_code)
        return None, None

# Obtém a matriz de distâncias e tempos
duration_matrix, distance_matrix = get_distance_matrix(localidades)

# Exibe a matriz de tempos
if duration_matrix:
    print("Matriz de tempos (segundos):")
    for row in duration_matrix:
        print(row)

# Exibe a matriz de distâncias
if distance_matrix:
    print("Matriz de distâncias (metros):")
    for row in distance_matrix:
        print(row)

Matriz de tempos (segundos):
[0, 494, 336, 256, 261, 388, 476, 158, 646, 586, 424]
[527, 0, 639, 498, 475, 513, 86, 599, 1051, 860, 820]
[240, 613, 0, 332, 466, 604, 680, 194, 594, 778, 279]
[289, 392, 386, 0, 404, 477, 464, 342, 799, 782, 568]
[282, 451, 509, 406, 0, 189, 429, 330, 694, 482, 508]
[375, 528, 593, 499, 172, 0, 501, 423, 694, 444, 534]
[591, 84, 703, 562, 487, 490, 0, 654, 1110, 872, 884]
[199, 598, 178, 321, 421, 520, 631, 0, 499, 687, 265]
[623, 997, 533, 746, 576, 616, 976, 577, 0, 530, 358]
[658, 854, 799, 801, 466, 463, 833, 664, 634, 0, 648]
[418, 779, 183, 498, 447, 546, 839, 372, 430, 645, 0]
Matriz de distâncias (metros):
[0, 5702, 2633, 1883, 2257, 4068, 5554, 1172, 5305, 6143, 3558]
[5239, 0, 7031, 5347, 4829, 6146, 944, 6287, 10707, 9859, 8704]
[1798, 7019, 0, 2704, 4006, 5562, 7078, 1452, 4711, 7566, 2536]
[2157, 4818, 3974, 0, 4023, 5270, 5690, 2660, 7650, 9054, 5647]
[2765, 5442, 4683, 4484, 0, 1941, 5608, 3222, 7137, 5371, 5022]
[4062, 6387, 6015, 5781, 1

In [8]:
def get_route(coordinates):
    """
    Obtém a rota real entre uma lista de coordenadas usando o OSRM.
    Retorna uma lista de coordenadas (latitude, longitude) que representam a rota.
    """
    url = "http://router.project-osrm.org/route/v1/driving/"
    coordinates_str = ";".join(coordinates)
    params = {
        "overview": "full",  # Retorna a geometria completa da rota
        "geometries": "geojson"  # Formato das coordenadas
    }
    response = requests.get(url + coordinates_str, params=params)
    if response.status_code == 200:
        data = response.json()
        if data['code'] == 'Ok':
            return data['routes'][0]['geometry']['coordinates']  # Lista de coordenadas (longitude, latitude)
    return None

In [9]:
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

# Passo 3: Resolver o problema de roteamento com a OR-Tools
# Número de funcionários (veículos)
num_funcionarios = 1  # Apenas 1 veículo

# Localização do depósito (origem: Centro de Curitiba)
depot = 0

# Cria o gerenciador de índices e o modelo de roteamento
manager = pywrapcp.RoutingIndexManager(len(localidades), num_funcionarios, depot)
routing = pywrapcp.RoutingModel(manager)

# Define os pesos para tempo e distância (ajuste conforme necessário)
peso_tempo = 0  # Peso para o tempo
peso_distancia = 1  # Peso para a distância

# Função de custo combinada (tempo e distância)
def combined_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    tempo = duration_matrix[from_node][to_node]  # Tempo entre from_node e to_node
    distancia = distance_matrix[from_node][to_node]  # Distância entre from_node e to_node
    return (peso_tempo * tempo) + (peso_distancia * distancia)  # Custo combinado

# Registra a função de custo combinada
combined_callback_index = routing.RegisterTransitCallback(combined_callback)

# Associa a função de custo ao modelo
routing.SetArcCostEvaluatorOfAllVehicles(combined_callback_index)

# Configura parâmetros de busca
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION  # Estratégia inicial
search_parameters.local_search_metaheuristic = routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH  # Metaheurística de busca local

search_parameters.time_limit.seconds = 60  # Tempo limite para otimização
search_parameters.log_search = True

# Resolve o problema
solution = routing.SolveWithParameters(search_parameters)

In [10]:
import folium
from folium.plugins import PolyLineTextPath

# Cria um mapa centrado no Centro de Curitiba
m = folium.Map(location=[-25.4284, -49.2733], zoom_start=12)

inv_localidades = [(lat,lon) for lon, lat in localidades]
inv_clientes = [(lat,lon) for lon, lat in clientes]
inv_origem = [(lat,lon) for lon, lat in origem]

# Adiciona a origem ao mapa
for i, (lat, lon) in enumerate(inv_origem):
    folium.Marker(
        [lat, lon],
        tooltip=f'Centro de Curitiba {i+1}',
        icon=folium.Icon(color='green')
    ).add_to(m)

# Adiciona os clientes ao mapa
for i, (lat, lon) in enumerate(inv_clientes):
    folium.Marker(
        [lat, lon],
        tooltip=f'Cliente {i+1}'
    ).add_to(m)

# Adiciona a rota real ao mapa com setas indicando o sentido
if solution:
    for vehicle_id in range(num_funcionarios):
        index = routing.Start(vehicle_id)
        route_coords = []
        route_output = ""
        route_time = 0
        route_distance = 0
        
        while not routing.IsEnd(index):
            next_index = solution.Value(routing.NextVar(index))
            current_node = manager.IndexToNode(index)
            next_node = manager.IndexToNode(next_index)
            # if routing.IsEnd(next_index):  # Se o próximo for o fim, sair do loop
            #     break
            current_node = manager.IndexToNode(index)
            lat, lon = map(float, inv_localidades[current_node])
            route_coords.append(f"{lon},{lat}")
            index = solution.Value(routing.NextVar(index))
            route_output += f'{current_node} -> '
            route_time += duration_matrix[current_node][next_node]
            route_distance += distance_matrix[current_node][next_node]  
            index = next_index

        # Adiciona a última localidade
        current_node = manager.IndexToNode(index)
        lat, lon = map(float, inv_localidades[current_node])
        route_coords.append(f"{lon},{lat}")

        route_output += f'{current_node}'
        total_distance_km = route_distance / 1000  # Convertendo metros para quilômetros
        total_time_minutes = route_time / 60  # Convertendo segundos para minutos
        # route_output += f'Distância total: {route_distance} metros\n'

        print(f'Rota Coords: {route_output}\nDistância total: {total_distance_km:.3f} km\nTempo total: {total_time_minutes:.3f} minutos\n')

        # Obtém a rota real entre os pontos
        route = get_route(route_coords)
        if route:
            # Converte as coordenadas para o formato [latitude, longitude]
            route_lat_lon = [[coord[1], coord[0]] for coord in route]

            # Cria a linha da rota
            linha = folium.PolyLine(
                route_lat_lon,
                color='blue',
                weight=1,
                opacity=1
            ).add_to(m)

            # Adiciona setas ao longo da linha para indicar o sentido
            PolyLineTextPath(
                linha,
                text='►',  # Seta Unicode
                repeat=True,
                offset=3,
                attributes={'fill': 'blue', 'font-size': '12', 'letter-spacing': '20'}
            ).add_to(m)


# Exibe o mapa
m.save('rota_otimizada_curitiba_com_arruamentos.html')
print("Mapa salvo como 'rota_otimizada_curitiba_com_arruamentos.html'")

Rota Coords: 0 -> 3 -> 1 -> 6 -> 4 -> 5 -> 9 -> 8 -> 10 -> 2 -> 7 -> 0
Distância total: 32.917 km
Tempo total: 57.033 minutos

Mapa salvo como 'rota_otimizada_curitiba_com_arruamentos.html'
